In [35]:
# 12. 윤곽선 검출
### 경계선을 연결한 선
### 흑백 이미지불러와서 -> 이진화 -> 윤곽선 찾고(findContours) -> 윤곽선 그리기(drawContours)
import cv2
maru_path = "images/maru.jpg"
maru_doll_path = "images/maru_doll.jpg"
vehicles_path = "images/vehicles.png"
playing_cards_path = "images/playing_cards.png"

####

In [14]:
## 12-1. 윤곽선 검출 기본
img = cv2.imread(maru_doll_path)
copied = img.copy()
copied_2 = img.copy()

gray = cv2.cvtColor(copied, cv2.COLOR_BGR2GRAY)  #그레이 스케일로 변환

ret, binary = cv2.threshold(gray, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)  #이진화

contours, hierachy = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)  #윤곽선 찾기
contours_2, hierachy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

cv2.drawContours(copied, contours, -1, (0,255,0), 2)  #윤곽선 그리기
cv2.drawContours(copied_2, contours_2, -1, (0,255,0), 2)

cv2.imshow("img", img)
cv2.imshow("contours", copied)
cv2.imshow("contours_2", copied_2)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
## 12-2. BoundingRect
### 윤곽선을 둘러싼 사각형 그리기

img = cv2.imread(maru_doll_path)
copied = img.copy()

gray = cv2.cvtColor(copied, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, -1, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) 

contours, hierachy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for contour in contours:
    x, y, width, height = cv2.boundingRect(contour)
    cv2.rectangle(copied, (x, y), (x + width, y + height), (255, 255, 0), 2)  #2 : 두께

cv2.imshow("contours1", copied)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
## 실습3. 순서대로 박스 표시(while True 사용)

### 의사코드 작성
#### 윤곽선을 찾고 : 그레이스케일 -> 이진화 -> findContours
#### 윤곽선의 인덱스를 선택해서 -> 트랙바 이용 -> while
#### 해당 인덱스의 bounding rect를 추출하고 -> cv2.boundingRect
#### 화면에 사각형을 그린다 -> cv2.rectangle

img = cv2.imread(vehicles_path)
copied = img.copy()

gray = cv2.cvtColor(copied, cv2.COLOR_BGR2GRAY)  #윤곽선을 찾고
ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)  #이진화
contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

name = "contour"  #윤곽선의 인덱스를 선택
trackbar_name = "index"
cv2.namedWindow(name)
cv2.createTrackbar(trackbar_name, name, 0, len(contours) - 1, lambda x:x) 


while True:
    index = cv2.getTrackbarPos(trackbar_name, name)
    copied = img.copy()

    x, y, w, h = cv2.boundingRect(contours[index])  #해당 인덱스의 bounding rect를 추출
    
    cv2.rectangle(copied, (x, y), (x + w, y + h), (0, 255, 0), 2)  #화면에 사각형을 그린다
    cv2.imshow(name, copied)

    
    if cv2.waitKey(1) == ord("q"):
        break

cv2.destroyAllWindows()

In [24]:
## 실습3. 순서대로 박스 표시(def 사용)

img = cv2.imread(vehicles_path)
copied = img.copy()

gray = cv2.cvtColor(copied, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

contours, hierachy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

def on_trackbar(val):
    copied = img.copy()
    x, y, w, h = cv2.boundingRect(contours[val])
    cv2.rectangle(copied, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("contour", copied)

name = "contour"
trackbar_name = "index"

cv2.namedWindow(name) 
cv2.createTrackbar(trackbar_name, name, 0, len(contours) - 1, on_trackbar) 

on_trackbar(0)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
## 12-3. contourArea(실습 3-1. 순서대로 박스 표시(큰 면적으로 윤곽선 잡히게), while True 사용)
### 윤곽선의 면적을 계산해줌

img = cv2.imread(vehicles_path)
copied = img.copy()

gray = cv2.cvtColor(copied, cv2.COLOR_BGR2GRAY)  #윤곽선을 찾고
ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

MIN_AREA = 1000  #윤곽선의 면적이 특정 크기 이상인 윤곽선을 새로운 리스트에 저장
filtered_contours = []
for contour in contours:
    if cv2.contourArea(contour) > MIN_AREA:
        filtered_contours.append(contour)


name = "contour"  #윤곽선의 인덱스를 선택
trackbar_name = "index"
cv2.namedWindow(name)
cv2.createTrackbar(trackbar_name, name, 0, len(filtered_contours) - 1, lambda x:x) 


while True:
    index = cv2.getTrackbarPos(trackbar_name, name)
    copied = img.copy()

    x, y, w, h = cv2.boundingRect(filtered_contours
[index])  #해당 인덱스의 bounding rect를 추출
    
    cv2.rectangle(copied, (x, y), (x + w, y + h), (0, 255, 0), 2)  #화면에 사각형을 그린다
    cv2.imshow(name, copied)

    
    if cv2.waitKey(1) == ord("q"):
        break

cv2.destroyAllWindows()

In [29]:
## 12-3. contourArea(실습 3-1. 순서대로 박스 표시(큰 면적으로 윤곽선 잡히게), def 사용)  -> ########## 집가서 수정하기 #########
img = cv2.imread(vehicles_path)
copied = img.copy()

gray = cv2.cvtColor(copied, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

contours, hierachy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

def on_trackbar(val):
    copied = img.copy()
    
    if cv2.contourArea(contours[val]) > 1000:
        x, y, w, h = cv2.boundingRect(contours[val])
        cv2.rectangle(copied, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("contour", copied)

name = "contour"
trackbar_name = "index"

cv2.namedWindow(name) 
cv2.createTrackbar(trackbar_name, name, 0, len(contours) - 1, on_trackbar) 

on_trackbar(0)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [52]:
## 실습4. 카드 하나씩 새 창에 표시
### 내 코드
img = cv2.imread(playing_cards_path)
copied = img.copy()

gray = cv2.cvtColor(copied, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

contours, hierachy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(img, (x,y), (x + w, y + h), (0, 255, 0), 2)

index = 1
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if w < 50 or h < 50:
        continue

    card = copied[y:y+h, x:x+w]

    window_name = f"Card {index+1}"
    cv2.imshow(window_name, card)
    index += 1

cv2.imshow("contour", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [53]:
## 실습4. 카드 하나씩 새 창에 표시
### 리더님 코드
img = cv2.imread(playing_cards_path)
copied = img.copy()

gray = cv2.cvtColor(copied, cv2.COLOR_BGR2GRAY)
ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

contours, hierachy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for idx, contour in enumerate(contours):
    if cv2.contourArea(contour) > 25000:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(img, (x,y), (x + w, y + h), (0, 255, 0), 2)
        crop = img[y:y+h, x:x+w]
        cv2.imshow(f"card{idx}", crop)

cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()